Import packages

In [92]:
import nltk
import os
import gensim
import scipy
import numpy as np
import collections
import pandas as pd
# from tqdm 
import tqdm
from operator import add
import  tensorflow as tf
from collections import OrderedDict
from utils_ import utility
from urllib import request
from itertools import chain
from functools import reduce
import gensim.downloader as gloader
from sklearn.metrics import f1_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, LayerNormalization, LSTM, Dense, Bidirectional, GRU, Embedding)

ModuleNotFoundError: No module named 'tensorflow.keras.models'

In [55]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [56]:
!pip install nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jyoti\AppData\Roaming\nltk_data...


[nltk_data]   Package punkt is already up-to-date!


True

In [57]:
utility.cleaning()

Current work directory: c:\Users\jyoti\Desktop\Nlp_project
Cleaned


In [58]:
url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"
Datasets="Datasets"
dependency_treebank="dependency_treebank.zip"
utility.downloadDataSet(url,Datasets,dependency_treebank)

Current work directory: c:\Users\jyoti\Desktop\Nlp_project
c:\Users\jyoti\Desktop\Nlp_project\Datasets\dependency_treebank.zip
c:\Users\jyoti\Desktop\Nlp_project\Datasets\dependency_treebank.zip
Download complete!
Extracting dataset... (it may take a while...)
<zipfile.ZipFile filename='c:\\Users\\jyoti\\Desktop\\Nlp_project\\Datasets\\dependency_treebank.zip' mode='r'>
Extraction completed!


In [59]:
def splitting(i):
    if i <= 100:
        return 'train'
    elif i <= 150:
        return 'val'
    else:
        return 'test'

In [60]:
dataset_name = "dependency_treebank"
debug = True

folder = os.path.join(os.getcwd(), "Datasets", dataset_name)
files=os.listdir(folder)
files.sort()
dataframe_rows = {'train': [], 'val':[], 'test':[]}


for i, filename in enumerate(files):
  file_path = os.path.join(folder, filename)
  try:
        if os.path.isfile(file_path):
              # open the file
              list1, list2 = [], []
              with open(file_path, mode='r', encoding='utf-8') as text_file:
              # read it and extract informations
                # text = text_file.read()
                for line in text_file:
                    tokens = line.split("\t")[:2]
                    # print(tokens)
                    if len(tokens) == 2:
                          list1.append(tokens[0])
                          list2.append(tokens[1])
                    if tokens[0] == '.' and splitting(i) != 'test':
                          dataframe_rows[splitting(i)].append({"tokens": list1, "tags": list2})
                          #print(dataframe_rows[:6])
                          list1, list2 = [], []
                if splitting(i) == 'test':
                    dataframe_rows[splitting(i)].append({"tokens": list1, "tags": list2})
                    list1, list2 = [], []
  except Exception as e:
        print('Failed to process %s. Reason: %s' % (file_path, e))
        sys.exit(0)

df_train = pd.DataFrame(dataframe_rows['train'])
df_val = pd.DataFrame(dataframe_rows['val'])
df_test = pd.DataFrame(dataframe_rows['test'])


In [61]:
df_train['split']='train'
df_test['split']='test'
df_val['split']='val'

In [62]:
df = pd.concat([df_train,df_test,df_val], ignore_index=True)
df

,tokens,tags,split
0,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ...",train
1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ...",train
2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP...",train
3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS...",train
4,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V...",train
...,...,...,...
3243,"[The, action, followed, by, one, day, an, Inte...","[DT, NN, VBN, IN, CD, NN, DT, NNP, NN, IN, PRP...",val
3244,"[In, New, York, Stock, Exchange, composite, tr...","[IN, NNP, NNP, NNP, NNP, JJ, NN, NN, ,, NNP, N...",val
3245,"[Mr., Edelman, declined, to, specify, what, pr...","[NNP, NNP, VBD, TO, VB, WP, VBD, DT, JJ, NNS, ...",val
3246,"['', He, added, ,, ``, This, has, nothing, to,...","['', PRP, VBD, ,, ``, DT, VBZ, NN, TO, VB, IN,...",val


Data Exploration

In [63]:
PAD, UNK, = '<pad>', '<unk>'
#<pad> all sequences should have same length. If the sequence is short then it will be padded
#<unk> means unknown token, used to replace rare word that didn't fir into the vocabulary
t2i_train, i2t = utility.build_vocabulary(df.loc[df['split'] == 'train', 'tags'], [PAD, UNK])
w2i_train, _ = utility.build_vocabulary(df.loc[df['split'] == 'train', 'tokens'])
# print(len(t2i_train))
# print(len(w2i_train))

In [64]:
t2i_val, i2t_val = utility.build_vocabulary(df.loc[df['split'] == 'val', 'tags'], [PAD, UNK])
#print(df.loc[df['split'] == 'train', 'tags'])
w2i_val, _ = utility.build_vocabulary(df.loc[df['split'] == 'val', 'tokens'])
# print(t2i_val)
# print(w2i_val)

In [65]:
t2i_test, i2t_test = utility.build_vocabulary(df.loc[df['split'] == 'test', 'tags'], [PAD, UNK])
#print(df.loc[df['split'] == 'train', 'tags'])
w2i_test, _ = utility.build_vocabulary(df.loc[df['split'] == 'test', 'tokens'])
# print(t2i_test)
# print(w2i_test)

In [66]:
tokens_fd=nltk.probability.FreqDist(w2i_train)
tokens_fd.most_common(10)

[('departments', 7478),
 ('17.3', 7477),
 ('bounce', 7476),
 ('leadership', 7475),
 ('flights', 7474),
 ('airline', 7473),
 ('27.1', 7472),
 ('anti-drug', 7471),
 ('supplemental', 7470),
 ('waived', 7469)]

In [67]:
embedding_dimension=100

embedding_model = utility.load_embedding_model('glove',embedding_dimension)

# allows "PAD" to have index zero, crucial for consistency
PAD, UNK, = '<pad>', '<unk>'
pad_unk_vec = np.zeros((2, embedding_dimension)) # pad
pad_unk_vec[1] = np.random.rand(embedding_dimension) # unk

In [68]:
df_val

,tokens,tags,split
0,"[Beauty, Takes, Backseat, To, Safety, on, Brid...","[NN, VBZ, NN, TO, NNP, IN, NNPS, NN, VBZ, IN, ...",val
1,"[But, there, 's, disagreement, over, how, to, ...","[CC, EX, VBZ, NN, IN, WRB, TO, VB, PRP, .]",val
2,"[Highway, officials, insist, the, ornamental, ...","[NN, NNS, VBP, DT, JJ, NNS, IN, JJR, NNS, VBP,...",val
3,"[But, other, people, do, n't, want, to, lose, ...","[CC, JJ, NNS, VBP, RB, VB, TO, VB, DT, NNS, PO...",val
4,"[``, The, primary, purpose, of, a, railing, is...","[``, DT, JJ, NN, IN, DT, NN, VBZ, TO, VB, DT, ...",val
...,...,...,...
1237,"[The, action, followed, by, one, day, an, Inte...","[DT, NN, VBN, IN, CD, NN, DT, NNP, NN, IN, PRP...",val
1238,"[In, New, York, Stock, Exchange, composite, tr...","[IN, NNP, NNP, NNP, NNP, JJ, NN, NN, ,, NNP, N...",val
1239,"[Mr., Edelman, declined, to, specify, what, pr...","[NNP, NNP, VBD, TO, VB, WP, VBD, DT, JJ, NNS, ...",val
1240,"['', He, added, ,, ``, This, has, nothing, to,...","['', PRP, VBD, ,, ``, DT, VBZ, NN, TO, VB, IN,...",val


In [69]:
df_train=df.loc[df['split']=='train']
#df_train
df_val=df.loc[df['split']=='val']
#df_val
df_test=df.loc[df['split']=='test']
#df_test

In [70]:
def text_2_categorical(df, w2i, t2i):
    """Converts DataFrame words to categorical for use in the RNNs"""
    df['tokens'] = df['tokens'].map(lambda s: [w2i.get(w.lower().strip(), w2i[UNK]) for w in s])
    df['tags'] = df['tags'].map(lambda s: [t2i.get(w.lower().strip()) for w in s])

In [71]:
df

,tokens,tags,split
0,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ...",train
1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ...",train
2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP...",train
3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS...",train
4,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V...",train
...,...,...,...
3243,"[The, action, followed, by, one, day, an, Inte...","[DT, NN, VBN, IN, CD, NN, DT, NNP, NN, IN, PRP...",val
3244,"[In, New, York, Stock, Exchange, composite, tr...","[IN, NNP, NNP, NNP, NNP, JJ, NN, NN, ,, NNP, N...",val
3245,"[Mr., Edelman, declined, to, specify, what, pr...","[NNP, NNP, VBD, TO, VB, WP, VBD, DT, JJ, NNS, ...",val
3246,"['', He, added, ,, ``, This, has, nothing, to,...","['', PRP, VBD, ,, ``, DT, VBZ, NN, TO, VB, IN,...",val


In [72]:
# pos tag vocabulary
p2i, i2p = utility.build_vocabulary(df_train['tags'], [PAD, UNK])

In [73]:
#Compute embeddings for terms out of vocabulary V1 (OOV1) of the training split 
voc1 = {e:i for i,e in enumerate(chain([PAD, UNK], embedding_model.vocab.keys()))}

##Build traning vocabulary

In [74]:
#Now let's build the training vocabulary
#Add embeddings to the vocabulary, so to obtain vocabulary V2=V1+OOV1

w2i_train, i2w_train = utility.build_vocabulary(df_train['tokens'])
voc2= utility.combine_vocabularies(voc1, w2i_train)
oov1=utility.check_OOV_terms(voc1, w2i_train)
print(f"Found {len(oov1)} oov words: {utility.getPercent(len(oov1), len(w2i_train)):.2}%")

Found 362 oov words: 4.8%


In [75]:
text_2_categorical(df_train, voc2, t2i_train)

C:\Users\jyoti\AppData\Local\Temp/ipykernel_13024/2031027438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['tokens'].map(lambda s: [w2i.get(w.lower().strip(), w2i[UNK]) for w in s])
C:\Users\jyoti\AppData\Local\Temp/ipykernel_13024/2031027438.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].map(lambda s: [t2i.get(w.lower().strip()) for w in s])


In [76]:
inv2 = {v:k for k,v in voc2.items()}
co_occ1 = utility.co_occurrence_count(df_train, inv2, sparse=True)
emb=utility.embedd_OOV_terms(embedding_model, oov1, co_occ1, voc2, inv2,embedding_dimension,rnd_OOV = False)
embedding_matrix_v2 = np.concatenate((pad_unk_vec, embedding_model.vectors.copy()))
#print(embedding_matrix_v2)

100%|██████████| 1958/1958 [00:03<00:00, 627.16it/s]


##Building Validation Vocabulary

In [77]:
w2i_val, _ = utility.build_vocabulary(df_val['tokens'])
voc3= utility.combine_vocabularies(voc2, w2i_val)
oov2=utility.check_OOV_terms(voc2, w2i_val)
print(f"Found {len(oov2)} oov words: {utility.getPercent(len(oov2), len(w2i_val)):.2}%\n")
text_2_categorical(df_val, voc3, t2i_val)

Found 189 oov words: 3.5%



C:\Users\jyoti\AppData\Local\Temp/ipykernel_13024/2031027438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['tokens'].map(lambda s: [w2i.get(w.lower().strip(), w2i[UNK]) for w in s])
C:\Users\jyoti\AppData\Local\Temp/ipykernel_13024/2031027438.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].map(lambda s: [t2i.get(w.lower().strip()) for w in s])


In [80]:
inv3 = {v:k for k,v in voc3.items()}
co_occ2 = utility.co_occurrence_count(df_val, inv3, sparse=True)
embeding_model = utility.embedd_OOV_terms(embedding_model, oov2, co_occ2, voc3, inv3,embedding_dimension,rnd_OOV = False)
embedding_matrix_v3 = np.concatenate((pad_unk_vec, embedding_model.vectors.copy()))

100%|██████████| 1242/1242 [00:01<00:00, 634.80it/s]


##Build test vocabulary

In [81]:
w2i_test, _ = utility.build_vocabulary(df_test['tokens'])
voc4= utility.combine_vocabularies(voc3, w2i_test)
oov3=utility.check_OOV_terms(voc3, w2i_test)
print(f"Found {len(oov3)} oov words: {utility.getPercent(len(oov3),len(w2i_test)):.2}%")
text_2_categorical(df_test, voc4, t2i_val)

Found 125 oov words: 3.7%


C:\Users\jyoti\AppData\Local\Temp/ipykernel_13024/2031027438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['tokens'].map(lambda s: [w2i.get(w.lower().strip(), w2i[UNK]) for w in s])
C:\Users\jyoti\AppData\Local\Temp/ipykernel_13024/2031027438.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].map(lambda s: [t2i.get(w.lower().strip()) for w in s])


In [82]:
inv4 = {v:k for k,v in voc4.items()}
co_occ3 = utility.co_occurrence_count(df_test, inv4, sparse=True)
embeding_model = utility.embedd_OOV_terms(embedding_model, oov3, co_occ3, voc4, inv4,embedding_dimension,rnd_OOV = False)
embedding_matrix_v4 = np.concatenate((pad_unk_vec, embedding_model.vectors.copy()))

100%|██████████| 48/48 [00:00<00:00, 52.53it/s]


In [84]:
df_train.drop('split',axis=1)
df_val.drop('split',axis=1)
df_test.drop('split',axis=1)

,tokens,tags
1958,"[2506, 1157, 1163, 1170, 18, 22, 2088, 49, 82,...","[5, 5, 16, 5, 32, 20, 32, 31, 38, 11, 2, 6, 5,..."
1959,"[16859, 148, 1020, 295, 2985, 1479, 1464, 12, ...","[5, 5, 5, 32, 11, 2, 12, 6, 9, 11, 2, 32, 5, 2..."
1960,"[7620, 953, 1020, 18, 22, 613, 470, 15, 203, 5...","[5, 5, 5, 32, 20, 32, 12, 6, 11, 6, 31, 11, 2,..."
1961,"[87, 3193, 3083, 470, 6494, 15, 5879, 14, 1344...","[2, 2, 12, 12, 32, 6, 12, 6, 5, 24, 9, 11, 2, ..."
1962,"[2, 3794, 5, 115381, 3, 51, 3273, 6, 598, 9, 7...","[9, 2, 6, 5, 24, 6, 23, 4, 14, 9, 2, 6, 9, 2, ..."
1963,"[2065, 6711, 1170, 18, 14, 73138, 785, 821, 13...","[5, 5, 5, 32, 6, 5, 5, 5, 5, 5, 3, 15, 23, 9, ..."
1964,"[10853, 4170, 5784, 18, 787, 400576, 3142, 21,...","[5, 5, 5, 32, 5, 5, 32, 6, 11, 2, 4, 14, 11, 1..."
1965,"[38582, 4076, 1020, 3, 9, 2885, 5, 400580, 776...","[5, 5, 5, 24, 9, 2, 6, 5, 5, 5, 10, 5, 5, 5, 1..."
1966,"[102798, 1020, 295, 9, 210377, 1052, 868, 5, 8...","[5, 5, 32, 9, 2, 11, 2, 6, 38, 28, 28, 24, 16,..."
1967,"[400584, 1020, 18, 22, 2872, 6, 257, 9, 210377...","[5, 5, 32, 20, 3, 4, 14, 9, 2, 11, 2, 6, 38, 2..."


#Create a baseline model, using a simple neural architecture

In [90]:
# base class model
class RNN(tf.keras.Model):
    def __init__(self, tag_vocab_size, sequence_len, embedding_matrix, batch_size):
        super(RNN, self).__init__()
        word_voc = embedding_matrix.shape[0]
        embedding_dim = embedding_matrix.shape[1]

        # input and embedding layers are shared among all the models
        self.input_layer = Input(batch_input_shape=(batch_size, sequence_len))
        self.embedding_layer = Embedding(word_voc, embedding_dim,weights=[embedding_matrix], trainable=False,mask_zero=True)


    def build(self):
        return Model(inputs=[self.input_layer], outputs=self(self.input_layer))

    def copy_weights(self, model):
        assert type(model).__name__ == type(self).__name__
        for layer, model_layer in zip(self.layers[1:], model.layers[1:]):
            layer.set_weights(model_layer.get_weights())

    def loss_function(self, y_true, predictions):
        mask = tf.math.logical_not(tf.math.equal(y, p2i[PAD])) # pad mask
        sce = tf.keras.losses.sparse_categorical_crossentropy(y_true, predictions, from_logits=False)
        mask = tf.cast(mask, dtype=sce.dtype)
        sce *= mask
        return tf.reduce_sum(sce)/tf.reduce_sum(mask)

    def update_metrics(self, y_true, predictions, loss_obj, acc_obj, f1_obj):

        # loss
        current_loss = self.loss_function(y_true, predictions)
        loss_obj.update_state(current_loss)

        p1 = tf.math.logical_or(tf.math.equal(y_true, p2i[PAD]),tf.math.equal(y, p2i['.']))
        p2 = tf.math.logical_or(tf.math.equal(y_true, p2i[',']),tf.math.equal(y, p2i[':']))
        punctuation = tf.math.logical_or(p1, p2)
        punctuation = tf.math.equal(y, p2i[PAD])
        sample_weights = tf.math.logical_not(punctuation)
        pred_np = np.argmax(predictions, axis=2)

        # accuracy
        acc_obj.update_state(y_true, pred_np, sample_weight=sample_weights)

        # f1 macro
        y_true_np = y.numpy().astype(np.int32)
        f1 = f1_score(y_true_np.flatten(), pred_np.flatten(), sample_weight=tf.reshape(sample_weights,[-1]),average='macro', zero_division=0, labels=[*p2i.values()])
        f1_obj.update_state(f1) 

AttributeError: module 'tensorflow.keras' has no attribute 'Model'

In [ ]:
# bidirectional lstm + dense

class BiLSTM(RNN):
    def __init__(self, tag_vocab_size, sequence_len, embedding_matrix, batch_size, lstm_dim):
        super(BiLSTM, self).__init__(tag_vocab_size, sequence_len, embedding_matrix, batch_size)
        self.lstm = Bidirectional(LSTM(lstm_dim, return_state=True, return_sequences=True))
        self.fc = Dense(tag_vocab_size, activation='softmax')
        
    def __call__(self, x):
        embedded = self.embedding_layer(x)
        lstm_o, lstmf_h, lstmf_c, lstmb_h, lstmb_c = self.lstm(embedded)
        output = self.fc(lstm_o)
        return output

In [ ]:
#GRU Model
class GRU(RNN):
    def __init__(self, tag_vocab_size, 
                 sequence_len, embedding_matrix, batch_size, gru_dim):
        super(GRU, self).__init__(tag_vocab_size, sequence_len, embedding_matrix, batch_size)
        self.gru = GRU(gru_dim, return_state=True, return_sequences=True)
        self.fc = Dense(tag_vocab_size, activation='softmax')
        
    def __call__(self, x):
        embedded = self.embedding_layer(x)
        gru_o, gru_sf, gru_sb = self.gru(embedded)
        print(gru_o,gru_sf,gru_sb)
        output = self.fc(gru_o)
        return output

In [ ]:
#BiGRU

# bidirectional lstm + dense ###################################################
class BiGRU(RNN):
    def __init__(self, tag_vocab_size, 
                 sequence_len, embedding_matrix, batch_size, gru_dim):
        super(BiGRU, self).__init__(tag_vocab_size, sequence_len, embedding_matrix, batch_size)
        self.gru = Bidirectional(GRU(gru_dim, return_state=True, return_sequences=True))
        self.fc = Dense(tag_vocab_size, activation='softmax')
        
    def __call__(self, x):
        embedded = self.embedding_layer(x)
        gru_o, gru_sf, gru_sb = self.gru(embedded)
        output = self.fc(gru_o)
        return output

In [ ]:
# bidirectional lstm + bidirectional lstm ######################################
class BiLSTM_BiLSTM(RNN): 
    def __init__(self, tag_vocab_size, sequence_len, embedding_matrix, batch_size, lstm_dim):
        super(BiLSTM_BiLSTM, self).__init__(tag_vocab_size, sequence_len, embedding_matrix, batch_size)
        self.lstm1 = Bidirectional(LSTM(
            lstm_dim, return_state=True, return_sequences=True))
        self.lstm2 = Bidirectional(LSTM(
            lstm_dim, return_state=True, return_sequences=True))
        self.fc = Dense(tag_vocab_size, activation='softmax')
        
    def __call__(self, x):
        embedded = self.embedding_layer(x)
        lstm_o, lstmf_h, lstmf_c, lstmb_h, lstmb_c = self.lstm1(embedded)
        lstm_o, lstmf_h, lstmf_c, lstmb_h, lstmb_c = self.lstm2(lstm_o)
        output = self.fc(lstm_o)
        return output

In [ ]:
# helper class to iterate the data
class DataIterator:
    def __init__(self, df, sequence_len, batch_size):

        #converting the tokens and tags to numpy
        self.X = df['tokens'].to_numpy()
        self.Y = df['tags'].to_numpy()

        #checking if the length of tokens and tags are equal
        assert self.X.shape[0] == self.Y.shape[0]
        self.num_sentence = self.X.shape[0]
        self.batch_size = batch_size


        # pad sentences
        self.data_list = []
        for i in range(self.num_sentence):
            m = np.zeros((sequence_len))
            n = np.zeros(sequence_len) + p2i[PAD]
            l = min(len(self.X[i]), sequence_len)
            m[:l] = self.X[i][:l]
            n[:l] = self.Y[i][:l]
            self.data_list.append((m, n))
        self.shuffle()


    def shuffle(self):
        self.current = 0
        random.shuffle(self.data_list)
        # batch the data
        num_batches = math.ceil(self.num_sentence/self.batch_size)
        self.batches_x = []
        self.batches_y = []
        for i in range(num_batches):
            batchx = []
            batchy = []
            for j in range(self.batch_size):
                if i*batch_size+j >= self.num_sentence:
                    break
                batchx.append(self.data_list[i*self.batch_size+j][0])
                batchy.append(self.data_list[i*self.batch_size+j][1])
            self.batches_x.append(np.array(batchx))
            self.batches_y.append(np.array(batchy))



    def __iter__(self):
        return self

    def __next__(self):
        if self.current >= len(self.batches_x):
            raise StopIteration
        x = self.batches_x[self.current]
        y = self.batches_y[self.current]
        x = tf.cast(x, tf.float32)
        y = tf.cast(y, tf.float32)
        self.current += 1
        return x,y



In [ ]:
def train_step_graph_fn(optimizer, model, x, y):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = model.loss_function(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return predictions


def val_step_graph_fn(model, x):
    predictions = model(x)
    return predictions   

# hyperparameters
sequence_len = 128
batch_size =   64
max_epochs = 100
learning_rate = 5e-3 
latent_dim =   64



#BiLSTM

In [ ]:

#pass the model which we want to run #BiLSTM
RNNModel = BiGRU

pos_vocab_size = len(p2i.keys())


# train model
model = RNNModel(pos_vocab_size, sequence_len, embedding_matrix_v2, batch_size, latent_dim)
model.build().summary()

# val model
val_model = RNNModel(pos_vocab_size, sequence_len, embedding_matrix_v3, batch_size, latent_dim)
val_model.build()


# best model
best_model = RNNModel(pos_vocab_size, sequence_len, embedding_matrix_v3, batch_size, latent_dim)
best_model.build()

In [ ]:
# Plot confusion matrix on all test set
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def confusion_matrix(y_acc,pred_acc):
  cm = confusion_matrix(y_acc, pred_acc)[1:,1:]
  cm = cm.astype('int')
  print(i2p[y])
  print(y_acc)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[i2p[y] for y in list(set(y_acc))][1:]) 

  fig = plt.figure(figsize=(25,25))
  ax = fig.add_subplot(111)
  disp.plot(include_values=True, ax=ax,values_format='.10g', cmap='inferno');
  # plt.show()

In [ ]:
# metrics
import random 
import math

train_loss_obj = tf.keras.metrics.Mean(name='train_loss')
train_f1_obj = tf.keras.metrics.Mean(name='train_f1')
train_acc_obj = tf.keras.metrics.Accuracy(name='train_accuracy')
val_loss_obj = tf.keras.metrics.Mean(name='val_loss')
val_f1_obj = tf.keras.metrics.Mean(name='val_f1')
val_acc_obj = tf.keras.metrics.Accuracy(name='val_accuracy')

# optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_data = DataIterator(df_train, sequence_len, batch_size)
val_data = DataIterator(df_val, sequence_len, batch_size)


# train loop
max_val_loss = math.inf
not_improving = 0
max_iter_not_improv = 2

errors = []
for epoch in range(max_epochs):
    train_loss_obj.reset_states()
    train_f1_obj.reset_states()
    train_acc_obj.reset_states()

    train_data.shuffle()
    errors = errors[:-10]
    for x,y in train_data:
        predictions = train_step_graph_fn(optimizer, model, x, y)
        #errors.append((x[np.argmax(predictions, axis = -1) != y], y, predictions))
        model.update_metrics(y, predictions, train_loss_obj, train_acc_obj, train_f1_obj)

    print("{}.  \t[TRAINING]\t  loss: {}  \t accuracy: {} \t f1-macro: {}".format(epoch, train_loss_obj.result(),train_acc_obj.result(),train_f1_obj.result()))

    # validation
    if epoch%5 == 4:
        val_loss_obj.reset_states()
        val_acc_obj.reset_states()
        val_f1_obj.reset_states()

        val_model.copy_weights(model)
        val_data.shuffle()
        for x,y in val_data:
            predictions = val_step_graph_fn(val_model, x)
            val_model.update_metrics(y, predictions, val_loss_obj, val_acc_obj, val_f1_obj)

        print("     \t[VALIDATION]\t   loss: {}  \t  accuracy: {} \t  f1-macro: {}".format( val_loss_obj.result(),val_acc_obj.result(),val_f1_obj.result()))

        # early stopping
        if val_loss_obj.result() < max_val_loss:
            best_model.copy_weights(val_model)
            max_val_loss = val_loss_obj.result()
            not_improving = 0
        else:
            not_improving += 1
            print("VALIDATION LOSS NOT IMPROVING, STRIKE:", not_improving,"!!")
            if not_improving >= max_iter_not_improv:
                print("Validation loss not improving for", max_iter_not_improv,
                      "successive computations.")
                break

In [ ]:
test_seq_len = 1500 
test_model = RNNModel(pos_vocab_size, test_seq_len, embedding_matrix_v4, batch_size, latent_dim)
test_model.build().summary()
test_model.copy_weights(best_model)

# run on the test set
test_loss_obj = tf.keras.metrics.Mean(name='test_loss')
test_f1_obj = tf.keras.metrics.Mean(name='test_f1')
test_acc_obj = tf.keras.metrics.Accuracy(name='test_accuracy')

test_data = DataIterator(df_test, test_seq_len, batch_size)

y_acc, pred_acc = [], []
for x, y in test_data:
    predictions = val_step_graph_fn(test_model, x)
    test_model.update_metrics(y, predictions, test_loss_obj, test_acc_obj, test_f1_obj)
    y_acc += list(y.numpy().flatten())
    if type(predictions) is tuple:
        pred_acc += list(predictions[0].numpy().flatten())
    else:                         # other models
        pred_acc += list(np.argmax(predictions, axis=2).flatten())

test_log = "\nTEST loss: {}  \t accuracy: {} \t f1-macro: {}\n".format(test_loss_obj.result(),test_acc_obj.result(),test_f1_obj.result())

print(test_log)
confusion_matrix(y_acc,pred_acc) 